In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import random

import tensorflow as tf
from tensorflow import keras

2023-05-09 18:02:09.313674: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Build data pipeline

## input and output variable list
- Note that ptend_t and ptend_q0001 are not in the output (mlo) netcdf files, but calculated real-time on a tf Dataset object.
- Variable list: https://docs.google.com/spreadsheets/d/1ljRfHq6QB36u0TuoxQXcV4_DSQUR0X4UimZ4QHR8f9M/edit#gid=0

In [2]:
# in/out variable lists
vars_mli = ['state_t','state_q0001','state_ps','pbuf_SOLIN', 'pbuf_LHFLX', 'pbuf_SHFLX']
vars_mlo = ['ptend_t','ptend_q0001','cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC','cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD']

## tf Dataset pipeline
- ref: https://www.noahbrenowitz.com/post/loading_netcdfs/
- ref: https://www.tensorflow.org/api_docs/python/tf/data/Dataset)

In [3]:
mli_mean = xr.open_dataset('./norm_factors/mli_mean.nc')
mli_min = xr.open_dataset('./norm_factors/mli_min.nc')
mli_max = xr.open_dataset('./norm_factors/mli_max.nc')
mlo_scale = xr.open_dataset('./norm_factors/mlo_scale.nc')

def load_nc_dir_with_generator(filelist:list):
    def gen():
        for file in filelist:
            
            # read mli
            ds = xr.open_dataset(file, engine='netcdf4')
            ds = ds[vars_mli]
            
            # read mlo
            dso = xr.open_dataset(file.replace('.mli.','.mlo.'), engine='netcdf4')
            
            # make mlo variales: ptend_t and ptend_q0001
            dso['ptend_t'] = (dso['state_t'] - ds['state_t'])/1200 # T tendency [K/s]
            dso['ptend_q0001'] = (dso['state_q0001'] - ds['state_q0001'])/1200 # Q tendency [kg/kg/s]
            dso = dso[vars_mlo]
            
            # normalizatoin, scaling
            ds = (ds-mli_mean)/(mli_max-mli_min)
            dso = dso*mlo_scale

            # stack
            #ds = ds.stack({'batch':{'sample','ncol'}})
            ds = ds.stack({'batch':{'ncol'}})
            ds = ds.to_stacked_array("mlvar", sample_dims=["batch"], name='mli')
            #dso = dso.stack({'batch':{'sample','ncol'}})
            dso = dso.stack({'batch':{'ncol'}})
            dso = dso.to_stacked_array("mlvar", sample_dims=["batch"], name='mlo')
            
            yield (ds.values, dso.values)

    sample = next(iter(gen()))

    return tf.data.Dataset.from_generator(
        gen,
        output_types=(tf.float64, tf.float64),
        output_shapes=((None,124),(None,128))
    )

## Instantiate tf.data.Dataset object here
- Dataset file size and dimensions: https://docs.google.com/document/d/1HgfZZJM0SygjWvSAJ5kSfql9aXUFkvLybL36p-vmdZc/edit

In [6]:
# for training
# First 5 days of each month for the first 6 years
f_mli1 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.000[123456]-*-0[12345]-*.nc')
f_mli2 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0007-01-0[12345]-*.nc')
f_mli = [*f_mli1, *f_mli2]
f_mli = f_mli[0:72*5] # for debugging only
random.shuffle(f_mli)
print(f'[TRAIN] Total # of input files: {len(f_mli)}')
print(f'[TRAIN] Total # of columns (nfiles * ncols): {len(f_mli)*384}')
tds = load_nc_dir_with_generator(f_mli)

# for validation
# First 5 days of each month for the following 2 years
f_mli1 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0007-0[23456789]-0[12345]-*.nc')
f_mli2 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0007-1[012]-0[12345]-*.nc')
f_mli3 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.000[89]-*-0[12345]-*.nc')
f_mli = [*f_mli1, *f_mli2, *f_mli3]
f_mli = f_mli[0:72*5] # for debugging only
random.shuffle(f_mli)
print(f'[VAL] Total # of input files: {len(f_mli)}')
print(f'[VAL] Total # of columns (nfiles * ncols): {len(f_mli)*384}')
tds_val = load_nc_dir_with_generator(f_mli)


#list(tds)
# for count_batch in tds.repeat().batch(10).take(1):
#     print(count_batch[0].numpy())
#count_batch[0].shape

[TRAIN] Total # of input files: 360
[TRAIN] Total # of columns (nfiles * ncols): 138240
[VAL] Total # of input files: 360
[VAL] Total # of columns (nfiles * ncols): 138240


# ML traning
- While 4 GPUs are available on the node, using multi GPUs (with 'tf.distribute.MirroredStrategy()' strategy) does not speed up training process.

In [7]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')]

In [14]:
# strategy = tf.distribute.MirroredStrategy()
# with strategy.scope():

# model params
input_length = 2*60 + 4
output_length_lin  = 2*60
output_length_relu = 8
output_length = output_length_lin + output_length_relu
n_nodes =128

# constrcut a model
input_layer    = keras.layers.Input(shape=(input_length,), name='input')
hidden_0       = keras.layers.Dense(n_nodes, activation='relu')(input_layer)
hidden_1       = keras.layers.Dense(n_nodes, activation='relu')(hidden_0)
output_pre     = keras.layers.Dense(output_length, activation='elu')(hidden_1)
output_lin     = keras.layers.Dense(output_length_lin,activation='linear')(output_pre)
output_relu    = keras.layers.Dense(output_length_relu,activation='relu')(output_pre)
output_layer   = keras.layers.Concatenate()([output_lin, output_relu])

model = keras.Model(input_layer, output_layer, name='Emulator')
model.summary()

# compile
model.compile(optimizer=keras.optimizers.Adam(), #optimizer=keras.optimizers.Adam(learning_rate=clr),
              loss='mse',
              metrics=['mse','mae','accuracy'])

Model: "Emulator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 124)]        0           []                               
                                                                                                  
 dense_15 (Dense)               (None, 128)          16000       ['input[0][0]']                  
                                                                                                  
 dense_16 (Dense)               (None, 128)          16512       ['dense_15[0][0]']               
                                                                                                  
 dense_17 (Dense)               (None, 128)          16512       ['dense_16[0][0]']               
                                                                                           

In [15]:
# batch is automatically set at 384, which is #ncol.
# to control batch size: use tf.data.Dataset.rebatch instead of batch_sizes option of fit method
# (https://www.tensorflow.org/api_docs/python/tf/data/Dataset#rebatch)
# e.g., model.fit(tds.rebatch(384*10), epochs=2)

model.fit(tds, validation_data=tds_val, epochs=5)

Epoch 1/5
360/360 [==============================] - 45s 122ms/step - loss: 0.0072 - mse: 0.0072 - mae: 0.0371 - accuracy: 0.9602 - val_loss: 0.0081 - val_mse: 0.0081 - val_mae: 0.0424 - val_accuracy: 0.9673
Epoch 2/5
360/360 [==============================] - 44s 122ms/step - loss: 0.0054 - mse: 0.0054 - mae: 0.0304 - accuracy: 0.9723 - val_loss: 0.0084 - val_mse: 0.0084 - val_mae: 0.0420 - val_accuracy: 0.9696
Epoch 3/5
360/360 [==============================] - 44s 124ms/step - loss: 0.0052 - mse: 0.0052 - mae: 0.0292 - accuracy: 0.9738 - val_loss: 0.0084 - val_mse: 0.0084 - val_mae: 0.0417 - val_accuracy: 0.9693
Epoch 4/5
360/360 [==============================] - 44s 124ms/step - loss: 0.0050 - mse: 0.0050 - mae: 0.0285 - accuracy: 0.9744 - val_loss: 0.0083 - val_mse: 0.0083 - val_mae: 0.0410 - val_accuracy: 0.9695
Epoch 5/5
360/360 [==============================] - 44s 123ms/step - loss: 0.0050 - mse: 0.0050 - mae: 0.0280 - accuracy: 0.9747 - val_loss: 0.0082 - val_mse: 0.0082 -